In [1]:
def pfound(rels, rel2prob, p_break=0.15):
    p_look = 1.0
    total = 0.0
    for r in rels:
        p_rel = rel2prob.get(r, 0.0)
        total += p_look * p_rel
        p_look = p_look * (1 - p_rel) * (1 - p_break)
    return total

# пример
rels = [2, 0, 1, 3, 0]
rel2prob = {0:0.0, 1:0.3, 2:0.6, 3:0.9}
print(pfound(rels, rel2prob, p_break=0.15))  # ≈ 0.84119

0.8414595


In [ ]:
# def get_probs(rel_matrix, prev_prob, p_break=0.15):
#     p_move_right = rel_matrix[0, 1] / (rel_matrix[1, 0] + rel_matrix[0, 1])
#     p_move_left_down = rel_matrix[1, 0] / (rel_matrix[1, 0] + rel_matrix[0, 1])
#     first_branch = (1 - rel_matrix[0, 0]) * (
#                     p_move_left_down * (1 - p_break) +
#                     p_move_right * (1 - p_break) * (1 - rel_matrix[0, 1]) * (1 - p_break))
#     p_move_left = rel_matrix[0, 0] / (rel_matrix[0, 0] + rel_matrix[1, 1])
#     p_move_right_down = rel_matrix[1, 1] / (rel_matrix[0, 0] + rel_matrix[1, 1])
#     second_branch = (1 - rel_matrix[0, 1]) * (
#                     p_move_right_down * (1 - p_break) +
#                     p_move_left * (1 - p_break) * (1 - rel_matrix[0, 0]) * (1 - p_break))
#     prob = prev_prob * (first_branch + second_branch)
#     return prob
def get_probs(rel_matrix, prev_prob, p_break=0.15):
    r1 = rel_matrix[0, 0]
    r2 = rel_matrix[0, 1]
    prob = prev_prob * ((1 - r1) * (1 - r2) * (1 - p_break))
    return prob

In [ ]:
def get_probs(rel_matrix, prev_prob, p_break=0.15):
    entrance_rel_1 = rel_matrix[0, 0] / (rel_matrix[0, 0] + rel_matrix[0, 1])
    entrance_rel_2 = rel_matrix[0, 1] / (rel_matrix[0, 0] + rel_matrix[0, 1])
    p_move_right = rel_matrix[0, 1] / (rel_matrix[1, 0] + rel_matrix[0, 1])
    p_move_left_down = rel_matrix[1, 0] / (rel_matrix[1, 0] + rel_matrix[0, 1])
    first_branch = (1 - rel_matrix[0, 0]) * (
                    p_move_left_down * (1 - p_break) +
                    p_move_right * (1 - p_break) * (1 - rel_matrix[0, 1]) * (1 - p_break))
    p_move_left = rel_matrix[0, 0] / (rel_matrix[0, 0] + rel_matrix[1, 1])
    p_move_right_down = rel_matrix[1, 1] / (rel_matrix[0, 0] + rel_matrix[1, 1])
    second_branch = (1 - rel_matrix[0, 1]) * (
                    p_move_right_down * (1 - p_break) +
                    p_move_left * (1 - p_break) * (1 - rel_matrix[0, 0]) * (1 - p_break))
    prob = prev_prob * (entrance_rel_1 * first_branch + entrance_rel_2 * second_branch)
    return prob

In [11]:
import numpy as np

rel_matrix = np.array([[0.2, 0.3],
                 [0.4, 0.1],
                 [0.6, 0.1],
                 [0.2, 0.7],
                 [0.01, 0.99],
                 [0.5, 0.5]])
prev_prob = 1.0

for i in range(rel_matrix.shape[0] - 1):
    m = np.vstack((rel_matrix[i], rel_matrix[i+1]))
    print(f"matrix {i}:\n{m}")
    prev_prob = get_probs(m, prev_prob=prev_prob, p_break=0.05)
    print("get_probs =>",prev_prob)
    print()

matrix 0:
[[0.2 0.3]
 [0.4 0.1]]
get_probs => 0.5955142857142857

matrix 1:
[[0.4 0.1]
 [0.6 0.1]]
get_probs => 0.33273186700408164

matrix 2:
[[0.6 0.1]
 [0.2 0.7]]
get_probs => 0.1321486658339354

matrix 3:
[[0.2  0.7 ]
 [0.01 0.99]]
get_probs => 0.034696901635051375

matrix 4:
[[0.01 0.99]
 [0.5  0.5 ]]
get_probs => 0.0004375082525160641



Создание матрицы переходов $P(s_i|s_j)$

In [1]:
from enum import IntEnum
import numpy as np

class Color(IntEnum):
    GRAY  = 0
    GREEN = 1
    YELLOW = 2
def generate_delivery(n_rows, n_cols, color_probs, rng=None):
    """
    Генерирует выдачу (сетку полок) с цветами на основе Enum Color.

    color_probs: dict[Color, float], например:
        {Color.GRAY: 0.5, Color.GREEN: 0.3, Color.YELLOW: 0.2}
    """
    if rng is None:
        rng = np.random.default_rng()

    # список всех цветов в порядке Enum
    colors = np.array(list(Color))          # [Color.GRAY, Color.GREEN, Color.YELLOW]
    probs = np.array([color_probs[c] for c in colors], dtype=float)

    # на всякий случай нормируем (если суммы слегка "плывут")
    probs /= probs.sum()

    # выбираем индексы цветов (0..len(colors)-1) по заданному распределению
    idx = rng.choice(len(colors), size=(n_rows, n_cols), p=probs)

    # layout в виде Enum
    layout_enum = colors[idx]              # dtype=object или Color

    return layout_enum

In [2]:
import numpy as np
from collections import deque

def create_p_look_matrix(P_look, delivery, start_pos=(0, 0)):
    """
    P_look: np.ndarray shape (n_colors, n_colors)
        P_look[c_from, c_to] — P(цвет_to | цвет_from)

    delivery: 2D-массив (n_rows x n_cols)
        коды цветов ячеек выдачи:
        - либо np.ndarray из IntEnum (Color),
        - либо np.ndarray из int (0..n_colors-1)

    start_pos: (r, c)
        координаты стартовой ячейки, для которой нет "предыдущей".
        Для неё можно считать P=1 (старт) или 0 — тут я ставлю 1.0.

    Возвращает:
        probs: 2D-массив той же формы, что delivery,
        где probs[r, c] = P(color(r,c) | color(parent(r,c)))
        по заданной матрице P_look.
    """
    delivery = np.asarray(delivery)
    n_rows, n_cols = delivery.shape

    # приведение Enum -> int (если delivery из Enum), либо int -> int
    colors = np.vectorize(int)(delivery)

    # задаём топологию — кто с кем сосед
    def neighbors(r, c):
        # крест: вверх, вниз, влево, вправо
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < n_rows and 0 <= nc < n_cols:
                yield nr, nc

    # 1. Строим "родителя" для каждой ячейки BFS-ом от start_pos
    parents = {start_pos: None}
    q = deque([start_pos])

    while q:
        r, c = q.popleft()
        for nr, nc in neighbors(r, c):
            if (nr, nc) not in parents:
                parents[(nr, nc)] = (r, c)
                q.append((nr, nc))

    # 2. Считаем P(ячейка | родитель) по цветам
    probs = np.zeros_like(colors, dtype=float)

    for (r, c), parent in parents.items():
        if parent is None:
            # стартовая ячейка — как хочешь, я ставлю 1.0
            probs[r, c] = 1.0
        else:
            pr, pc = parent
            c_from = colors[pr, pc]   # цвет родителя
            c_to   = colors[r, c]     # цвет текущей ячейки
            probs[r, c] = P_look[c_from, c_to]

    return probs


Подсчет матрицы переходов

In [6]:
def get_transition_matrix(rel_matrix, p_look_matrix, p_break = 0.15):
    # 0_0 0_1
    # 1_0 1_1
    #"S0", "S1", "S2", "S3", "S0B", "S1B", "S2B", "S3B", "S0D", "S1D", "S2D", "S3D",  "None"
    transition_matrix = np.zeros((13, 13))
    #first row
    p_move_right = p_look_matrix[0, 1] / (p_look_matrix[1, 0] + p_look_matrix[0, 1])
    p_move_left_down = p_look_matrix[1, 0] / (p_look_matrix[1, 0] + p_look_matrix[0, 1])
    transition_matrix[0, 0] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_left_down # P(s0|s0)
    transition_matrix[0, 3] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_right * (1 - rel_matrix[0, 1]) * (1 - p_break) # P(s3|s0)
    transition_matrix[0, 4] = rel_matrix[0, 0] # P(s0b|s0)
    transition_matrix[0, 7] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_right * rel_matrix[0, 1] # P(s3b|s0)
    transition_matrix[0, 8] = (1 - rel_matrix[0, 0]) * p_break # P(s0d|s0)
    transition_matrix[0, 11] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_right * (1 - rel_matrix[0, 1]) * p_break # P(s3d|s0)

    #second_row
    transition_matrix[1, 0] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_left_down # P(s0|s1)
    transition_matrix[1, 3] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_right * (1 - rel_matrix[0, 1]) * (1 - p_break) # P(s3|s1)
    transition_matrix[1, 4] = rel_matrix[0, 0] # P(s0b|s1)
    transition_matrix[1, 7] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_right * rel_matrix[0, 1] # P(s3b|s1)
    transition_matrix[1, 8] = (1 - rel_matrix[0, 0]) * p_break # P(s0d|s1)
    transition_matrix[1, 11] = (1 - rel_matrix[0, 0]) * (1 - p_break) * p_move_right * (1 - rel_matrix[0, 1]) * p_break # P(s3d|s1)

    #third row
    p_move_left = p_look_matrix[0, 0] / (p_look_matrix[0, 0] + p_look_matrix[1, 1])
    p_move_right_down = p_look_matrix[1, 1] / (p_look_matrix[0, 0] + p_look_matrix[1, 1])
    transition_matrix[2, 1] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_left * (1 - rel_matrix[0, 0]) * (1 - p_break) #P(s1|s2)
    transition_matrix[2, 2] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_right_down # P(s2|s2)
    transition_matrix[2, 5] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_left * rel_matrix[0, 0] #P(s1b|s2)
    transition_matrix[2, 6] = rel_matrix[0, 1] # P(s2b|s2)
    transition_matrix[2, 9] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_left * (1 - rel_matrix[0, 0]) * p_break # P(s1d|s2)
    transition_matrix[2, 10] = (1 - rel_matrix[0, 1]) * p_break # P(s2d|s2)

    #fourth row
    transition_matrix[3, 1] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_left * (1 - rel_matrix[0, 0]) * (1 - p_break) #P(s1|s3)
    transition_matrix[3, 2] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_right_down # P(s2|s3)
    transition_matrix[3, 5] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_left * rel_matrix[0, 0] #P(s1b|s3)
    transition_matrix[3, 6] = rel_matrix[0, 1] # P(s2b|s3)
    transition_matrix[3, 9] = (1 - rel_matrix[0, 1]) * (1 - p_break) * p_move_left * (1 - rel_matrix[0, 0]) * p_break # P(s1d|s3)
    transition_matrix[3, 10] = (1 - rel_matrix[0, 1]) * p_break # P(s2d|s3)

    #rest_rows
    transition_matrix[4:, 12] = 1.0  # P(None|sX) = 1.0 for all buying and break states

    return transition_matrix

In [60]:
P_look = np.array([
    #   GRAY   GREEN  YELLOW
    [ 0.6,  0.3,  0.1 ],  # GRAY
    [ 0.2,  0.6,  0.2 ],  # GREEN
    [ 0.3,  0.3,  0.4 ],  # YELLOW
])
color_probs = {
    Color.GRAY: 0.5,
    Color.GREEN: 0.3,
    Color.YELLOW: 0.2,
}

layout_enum, layout_ids = generate_delivery(
    n_rows=6,
    n_cols=2,
    color_probs=color_probs
)
print("Generated layout (Enum):")
print(layout_enum)

Generated layout (Enum):
[[0 0]
 [0 0]
 [0 0]
 [2 2]
 [2 0]
 [1 0]]


In [61]:
P_look = np.array([
    #   GRAY   GREEN  YELLOW
    [ 0.6,  0.3,  0.1 ],  # GRAY
    [ 0.2,  0.6,  0.2 ],  # GREEN
    [ 0.3,  0.3,  0.4 ],  # YELLOW
])
p_look_matrix = create_p_look_matrix(P_look, layout_enum)
p_look_matrix

array([[1. , 0.6],
       [0.6, 0.6],
       [0.6, 0.6],
       [0.1, 0.4],
       [0.4, 0.3],
       [0.3, 0.2]])

In [10]:
from enum import IntEnum


def demo():
    # Пример матрицы релевантностей (вероятность купить при просмотре)
    # 0-я строка — верхняя полка, 1-я — нижняя (условно); колонки — левая/правая карточки
    rel_matrix = np.array([[0.2, 0.3],
                 [0.4, 0.1],
                 [0.6, 0.1],
                 [0.2, 0.7],
                 [0.01, 0.99],
                 [0.5, 0.5]]
    )

    P_look = np.array([
        #   GRAY   GREEN  YELLOW
        [ 0.6,  0.3,  0.1 ],  # GRAY
        [ 0.2,  0.6,  0.2 ],  # GREEN
        [ 0.3,  0.3,  0.4 ],  # YELLOW
    ])
    color_probs = {
        Color.GRAY: 0.5,
        Color.GREEN: 0.3,
        Color.YELLOW: 0.2,
    }

    layout_enum = generate_delivery(
        n_rows=6,
        n_cols=2,
        color_probs=color_probs
    )
    p_look_matrix = create_p_look_matrix(P_look, layout_enum)
    T = get_transition_matrix(rel_matrix, p_look_matrix, p_break=0.15)

    np.set_printoptions(precision=3, suppress=True)
    # print("\nМатрица переходов T (13x13) =\n", T)

    row_sums = T.sum(axis=1)
    print("\nСуммы по строкам:\n", row_sums)

    state = np.array([0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    state = state @ T
    for i in range(1, rel_matrix.shape[0] - 1):
        m = np.vstack((rel_matrix[i], rel_matrix[i+1]))
        print(f"matrix {i}:\n{m}")
        T = get_transition_matrix(m, p_look_matrix[i:i+2, :], p_break=0.05)
        state = state @ T
        print(f"\nРаспределение после {i} шага:\n", state)
        print()
    

In [11]:
demo()


Суммы по строкам:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
matrix 1:
[[0.4 0.1]
 [0.6 0.1]]

Распределение после 1 шага:
 [0.084 0.026 0.136 0.072 0.119 0.018 0.021 0.008 0.009 0.001 0.01  0.004
 0.491]

matrix 2:
[[0.6 0.1]
 [0.2 0.7]]

Распределение после 2 шага:
 [0.021 0.034 0.089 0.018 0.066 0.054 0.021 0.002 0.002 0.002 0.009 0.001
 0.681]

matrix 3:
[[0.2  0.7 ]
 [0.01 0.99]]

Распределение после 3 шага:
 [0.014 0.012 0.015 0.008 0.011 0.003 0.075 0.019 0.002 0.001 0.002 0.
 0.838]

matrix 4:
[[0.01 0.99]
 [0.5  0.5 ]]

Распределение после 4 шага:
 [0.012 0.    0.    0.    0.    0.    0.023 0.012 0.001 0.    0.    0.
 0.951]



In [ ]:
alpha = 1
n = 6
P = np.array([
    [alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, 0.0],  
    [0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0],  
    [0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0], 
    [alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, alpha/n, alpha/n, 0.0, 0.0, 0.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 
])
# Начальное распределение: на 100% в сумме у трех состояний где он выжил
pi = np.array([0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 

# print("Шаг 0:", pi, "=>", dict(zip(states, pi)))
print("Шаг 0:", pi, np.sum(pi))

# Делаем 10 шагов: pi_{t+1} = pi_t * P
for t in range(1, 20):
    pi = pi @ P      # умножение вектора-строки на матрицу переходов
#   print(f"Шаг {t}:", pi,'\n',  "=>", dict(zip(states, pi)))
    print(f"Шаг {t}:", pi, np.sum(pi))